## Lec 6

### Logisic regression 0과 1로 수렴하는 함수 이용

### Multinomial classificaion - 여러개의 클래스를 예측할 때 유용

#### 두개 이상을 분류 binary classificaion 여러개로 구현 가능

#### matrix로 표현 합쳐서 한번에 곱샘으로 연산

#### sigmoid 사용해서 0과 1사이의 값으로 표현되게함 -> softmax 사용

#### softmax를 사용한후 one hot 인코딩으로 최댓값 찾음
#### softmax function : scores -> probabilities

#### cross entropy cost function 예측 맞으면 적은 값 틀리면 매우 큰값을 줘서 벌?줌

### Logisic cost vs cross entropy
#### 결과는 같음 ㅋ

#### 마찬가지로 gradient descent algorithm 사용 0 -> 최소값 탐색

### Lab 6-1
#### tensorflow hypothesis = tf.nn.softmax() 라이브러리 제공
#### 값은 logit 넣어줌(matrix)
#### lossfunction - y*log(yhat)
#### optimizer gradientDescentOptimizer 사용

In [6]:
import tensorflow as tf
x_data = [[1, 2, 1, 1], [2, 1, 3, 2], [3, 1, 3, 4], [4, 1, 5, 5], [1, 7, 5, 5], 
                                                        [1, 2, 5, 6], [1, 6, 6, 6], [1, 7, 7, 7]]
y_data = [[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0]]
# y value one hot encoding 사용
X = tf.placeholder("float", [None, 4]) # element 4개
Y = tf.placeholder("float", [None, 3])
nb_classes = 3

W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

with tf.Session() as sess:
   sess.run(tf.global_variables_initializer())

   for step in range(2001): # loop돌면서 optimizer
       sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
       if step % 200 == 0:
           print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}))
       


0 2.0059505
200 0.54064214
400 0.4266229
600 0.32803386
800 0.2562497
1000 0.22966684
1200 0.20829716
1400 0.1905615
1600 0.17557058
1800 0.16272177
2000 0.15158404


### Lab 6-2
#### 동물 분류 학습

In [15]:
import tensorflow as tf
import numpy as np
# 읽어온 테이블 마지막 열 빼고 x
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

nb_classes = 7  # 0 ~ 6

X = tf.placeholder(tf.float32, [None, 16]) # x가 16개
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6

Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot으로 바꿔줌
# one hot 사용시 한차원을 더함
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes]) #다시 2차원으로 reshape해줌


W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)


cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1) # 0~6 사이값으로 만들어줌
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# Launch graph
with tf.Session() as sess:
   sess.run(tf.global_variables_initializer())

   for step in range(2000):
       sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
       if step % 100 == 0:
           loss, acc = sess.run([cost, accuracy], feed_dict={
                                X: x_data, Y: y_data})
           print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(
               step, loss, acc))

   
   pred = sess.run(prediction, feed_dict={X: x_data})
   
   for p, y in zip(pred, y_data.flatten()):
       print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))


Step:     0	Loss: 9.564	Acc: 4.95%
Step:   100	Loss: 0.947	Acc: 78.22%
Step:   200	Loss: 0.610	Acc: 84.16%
Step:   300	Loss: 0.442	Acc: 89.11%
Step:   400	Loss: 0.337	Acc: 90.10%
Step:   500	Loss: 0.269	Acc: 93.07%
Step:   600	Loss: 0.221	Acc: 93.07%
Step:   700	Loss: 0.187	Acc: 94.06%
Step:   800	Loss: 0.161	Acc: 99.01%
Step:   900	Loss: 0.142	Acc: 99.01%
Step:  1000	Loss: 0.126	Acc: 99.01%
Step:  1100	Loss: 0.114	Acc: 99.01%
Step:  1200	Loss: 0.104	Acc: 100.00%
Step:  1300	Loss: 0.096	Acc: 100.00%
Step:  1400	Loss: 0.088	Acc: 100.00%
Step:  1500	Loss: 0.082	Acc: 100.00%
Step:  1600	Loss: 0.077	Acc: 100.00%
Step:  1700	Loss: 0.072	Acc: 100.00%
Step:  1800	Loss: 0.068	Acc: 100.00%
Step:  1900	Loss: 0.065	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 3 True